# Assignment 3: A\*, IDS, and Effective Branching Factor

## By Vignesh Madharapakkam Pagadala

## Summary

This Notebook contains the implementation of the following search algorithms:
* The Recursive Best-First Search implementation of A\*.
* Iterative Deepening Search

Additionally, we define a function to calculate the Effective Branching Factor for each algorithm, and use it to contrast the performance of the algorithms when applied on different eight-tile puzzle examples. We also try A\* with different heuristic functions and observe the results.

## Contents
1. <a href = '#1'>Functions Implemented</a><a id = '1'></a>
2. <a href = '#astar'>A\* Search</a>
    * Overview
    * Algorithm Definition
    * Implementation <br><br>
3. <a href = '#heuristic'>Heuristic Functions and Admissibility</a>
4. <a href = '#1'>Iterative Deepening Search</a>
5. <a href = '#1'>Effective Branching Factor</a>
6. <a href = '#1'>Comparision of Results</a>


## 1. Functions Implemented
The implementation for the following functions have been included in this Notebook:
* <a href = '#actionsF_8p'>actionsF_8p</a>: Takes a state, and returns valid actions from that state.
* <a href = '#takeActionF_8p'>takeActionF_8p</a>: Takes in a state and a valid action, and applies the action onto the state.
* <a href = '#findBlank_8p'>findBlank_8p</a>: Finds the location of the blank in an eight-tile puzzle.
* <a href = '#printPath_8p'>printPath_8p</a>: Prints out the path taken by the algorithm in a readable format.
* <a href = '#printState_8p'>printState_8p</a>: Prints out the state of the eight puzzle in a readable format.
* <a href = '#matrix_to_list_8p'>matrix_to_list_8p</a>: Converts the 2-D eight-puzzle index to 1-D.
* <a href = '#list_to_matrix_8p'>list_to_matrix_8p</a>: Converts 1-D eight-puzzle index to 2-D.
* <a href = '#swap'>swap</a>: Swaps the values between two positions of the eight-puzzle. 
* <a href = '#goalTestF_8p'>goalTestF_8p</a>: Takes in a state and checks whether if it's the goal state.<a id = 'astar'></a>
* <a href = '#depthLimitedSearch'>depthLimitedSearch</a>: Function which performs Depth-Limited Search.
* <a href = '#iterativeDeepeningSearch'>iterativeDeepeningSearch</a>: Function which performs Iterative-Deepening Search.
* <a href = '#h1_8p'>h1_8p</a>: Heuristic function which returns zero for all states. 
* <a href = '#h2_8p'>h2_8p</a>: Heuristic function which returns the Manhattan Distance that the blank is from it's goal position.
* <a href = '#h3_8p'>h3_8p</a>: Heuristic function which returns the Manhattan Distance of all tiles (textbook version).
* <a href = '#ebf'>ebf</a>: Given the number of nodes of a tree and a depth, returns the Effective Branching Factor for that tree.
* <a href = '#runExperiment'>runExperiment</a>: Function to compare results between different algorithms and examples.


## 2. A\* Search

### Overview

According to Wikipedia, A\* is defined as "*a computer algorithm that is widely used in pathfinding and graph traversal, the process of plotting an efficiently directed path between multiple points, called nodes*". A\* is an **'informed' search** methodology, wherein, the algorithm makes use of some information about where the goal might be located, resulting in a more directed and focused search. <br>

<img src = "https://upload.wikimedia.org/wikipedia/commons/6/60/A%2A_Search_Example_on_North_American_Freight_Train_Network.gif", height = 400, width = 400>
Source: Wikimedia Commons (https://commons.wikimedia.org/wiki/File:A*_Search_Example_on_North_American_Freight_Train_Network.gif)

A\* decides on which node to expand next by using two parameters - g(n) and h(n). g(n) denotes the cost to reach the node n, whereas h(n) is the the estimated cost of the cheapest path from the node n to the goal. The algorithm uses the sum of these two values f(n), to determine which node to expand next. It is most plausible to go for the node with the least f(n) value, since we wish to minimize the path cost.

<center>*f(n) = g(n) + h(n)*<center>

In the above equation, h(n) is what is called the **heuristic function**. We shall discuss more about this later in the Notebook.  

### Algorithm Definition

In this Notebook, we shall look into the Recursive Best-First Search implementation of A\*. The algorithmic definition below is from our textbook authors.

Source: Figure 3.26, 'Artificial Intelligence - A Modern Approach' by Russell and Norvig

### Implementation <a id = 'actionsF_8p'></a>

We firstly define some functions which shall be put to use when solving the eight-tile puzzle problem. Click <a href = '#as'>here</a> to get directly to the A* implementation.

#### actionsF_8p:
Takes a list which is the state of the puzzle, as a parameter. Returns all possible valid actions from given state, as a list of tuples, with each tuple containing the action (which is a string containing either 'left', 'right', 'up' or 'down') and the single-step cost, which is assumed to be 1 for all moves. <br>
**Parameters**: 
1. *state*: The state of the 8-puzzle, in the form of a list with 9 elements - the numbers 1 to 8, and a 0 (which represents the blank in the 8-puzzle).<br>

**Return value**: This function returns a list of all allowable actions from the given state. This is returned in the form of a list.

In [2]:
# THE ACTIONSF FUNCTION GENERATES ALL POSSIBLE VALID ACTIONS FROM A GIVEN STATE.

def actionsF_8p(state):

    # FIRSTLY, WE DETERMINE WHERE THE BLANK IS LOCATED
    blank = findBlank_8p(state)
    
    # LET'S INITIALIZE THE VALID ACTIONS LIST
    validActions = []

    # NOW WE USE A SET OF CONDITIONALS AND KEEP APPENDING TO validActions
    
    # FOR GOING LEFT, CONDITION IS (Y != 0)
    if blank[1] != 0:
        validActions.append(('left', 1))
    # FOR GOING RIGHT, CONDITION IS (Y != 2)
    if blank[1] != 2:
        validActions.append(('right', 1))
    # FOR GOING UP, CONDITION IS (X != 0)
    if blank[0] != 0:
        validActions.append(('up', 1))
    # FOR GOING DOWN, CONDITION IS (X != 2)
    if blank[0] != 2:
        validActions.append(('down', 1))

    return validActions

#### takeActionF_8p:<a id = 'takeActionF_8p'></a>
Takes in *state* and an action. Applies this action onto the state, and returns the new state generated along with the single-step cost of 1.<br>
Parameters: 
1. *state*: The state of the 8-puzzle, in the form of a list with 9 elements - the numbers 1 to 8, and a 0 (which represents the blank in the 8-puzzle).
2. *action*: The action which is to be taken from the current state. *action* is a string whose value is either 'left', 'right', 'up' or 'down', each representing the direction the blank is to be moved.<br>

Return value: A tuple with two elements. The first element the state of the puzzle, after applying the indicated action on the given state (both which are specified in the parameters). This is returned as a list containing 9 elements. The second tuple element is the single-step cost which in this case is 1. 

In [3]:
# This function takes in a state and an action, and applies this action to the state. Returns the new state and a step cost to get to state (1).
import copy
def takeActionF_8p(state, action):
    # DETERMINE BLANK LOCATION
    blank = findBlank_8p(state)
    state2 = copy.copy(state)
    # MOVING LEFT - SWAP BLANK WITH ELEMENT TO THE LEFT.
    if action[0] == 'left':
        swap(state2, blank[0], blank[1], blank[0], blank[1] - 1)

    # MOVING RIGHT - SWAP BLANK WITH ELEMENT TO THE RIGHT.
    if action[0] == 'right':
        swap(state2, blank[0], blank[1], blank[0], blank[1] + 1)

    # MOVING UP - SWAP BLANK WITH ELEMENT ABOVE.
    if action[0] == 'up':
        swap(state2, blank[0], blank[1], blank[0] - 1, blank[1])

    # MOVING DOWN - SWAP BLANK WITH ELEMENT BELOW.
    if action[0] == 'down':
        swap(state2, blank[0], blank[1], blank[0] + 1, blank[1])

    return state2, 1

#### findBlank_8p<a id = 'findBlank_8p'></a>
Takes in the state of the 8-puzzle and returns the position of the blank.
Parameters: 
1. *state*: The state of the 8-puzzle, in the form of a list with 9 elements - the numbers 1 to 8, and a 0 (which represents the blank in the 8-puzzle).

Return value: The location of the blank, in the form of a tuple (x, y), where x is the x-index and y is the y-index, of the two-dimensional representation of the 8-puzzle's state. If the blank isn't found, then the string 'Blank not found!' is returned. 

In [4]:
# FUNCTION TO FIND THE BLANK IN A GIVEN STATE

def findBlank_8p(state):
    ctr = 0
    for i in state: # ITERATE THROUGH 'STATE' LIST AND ONCE BLANK IS FOUND, RETURN IT'S 2-DIMENSIONAL INDEX.
        if i == 0:
            return list_to_matrix_8p(ctr)
        ctr += 1
    return 'Blank not found!'

#### printPath_8p <a id = 'printPath_8p'></a>
Takes the start state, goal state and path (which is a list containing all the states gone through to get from start to goal), and prints out the entire solution path in a readable format. 
Parameters:
1. *startState*: A list with 9 elements which represents the starting state of the 8-puzzle.
2. *goalState*: A list with 9 elements which represents the goal state of the 8-puzzle.
3. *path*: Contains the solution path from *startState* to *goalState*.

Return value: Returns *None*.

In [5]:
def printPath_8p(startState, goalState, path):
    
    if path == 'cutoff' or path == 'failure':
        print(path)
    else:
        # FIND PATH LENGTH
        l = len(path)
        #print("The path from %s to %s is %d nodes long." % (printState_8p(startState), printState_8p(goalState), l))
        print("The path from \n")
        printState_8p(startState)
        print("\nto\n")
        printState_8p(goalState)
        print("\nis %d nodes long." % l)
        print()
        print()
        for p in path:
            printState_8p(p)
            print()

#### printState_8p <a id = 'printState_8p'></a>
Prints out the state of the eight-puzzle in a comprehensible way.
Parameters:
1. *state*: The state of the 8-puzzle, in the form of a list with 9 elements - the numbers 1 to 8, and a 0 (which represents the blank in the 8-puzzle). 

Return value: Returns *None*.

In [6]:
def printState_8p(state):
    ctr = 0
    # PRINTING THE ELEMENTS OF THE STATE IN A READABLE, 2-DIMENSIONAL FORMAT
    for i in range(3):
        for j in range(3):
            if state[ctr] == 0:
                print(' ', end = ' ')
            else:
                print(state[ctr], end=' ')
            ctr += 1
        print()

#### matrix_to_list_8p <a id = 'matrix_to_list_8p'></a>
It would be helpful to visualize the 8-puzzle as a 2-dimensional 3 X 3 board. This raises the need to index this 3 X 3 space, but it is also required to index the 1-dimensional list *state* in which the state of the puzzle is stored. This calls for the need to have a function which converts the 2-D indices into 1-D, which is what *matrix_to_list_8p* does.
Parameters: 
1. *x*: x-index of the 3 X 3 8-puzzle
2. *y*: y-index of the 3 X 3 8-puzzle

Return value: The equivalent 1-dimensional index of the puzzle's state. If index doesn't exist, the string 'Index does not exist!' is returned.

In [7]:
# THIS FUNCTION TAKES IN X INDEX AND Y INDEX VALUES, AND RETURNS THE SIMPLE-LIST INDEX

def matrix_to_list_8p(x, y):
    counter = 0
    for i in range(3): # SINCE IT'S A 3 X 3 PUZZLE
        for j in range(3):
            if i == x and j == y:
                return counter
            counter += 1
    return 'Index does not exist!'

#### list_to_matrix_8p <a id = 'list_to_matrix_8p'></a>
Similar to why we defined the *matrix_to_list_8p* function, *list_to_matrix_8p* takes in the 1-dimensional index of *state* and returns the equivalent 2-dimensional index.
Parameters: 
1. *x*: index of the *state* list

Return value: The equivalent 2-dimensional index of the puzzle's state, returned as a (x,y) tuple. If index doesn't exist, the string 'Index does not exist!' is returned.

In [13]:
# FUNCTION WHICH TAKES IN A SIMPLE-LIST INDEX AND RETURNS X, Y VALUES

def list_to_matrix_8p(x):
    counter = 0
    for i in range(3):
        for j in range(3):
            if counter == x:
                return i, j # RETURNS A TUPLE CONTAINING X AND Y INDEX VALUES
            counter += 1
    return 'Index does not exist!'

#### swap <a id = 'swap'></a>
It takes in the state of the puzzle, and the 2-dimensional indices of two elements in the list. It then swaps the position of these two elements in the *state* list. 
Parameters:
1. *state*: State of the puzzle, represented as a list with 9 elements.
2. *x1*: An integer which represents the x-index of the first element.
3. *y1*: An integer which represents the y-index of the first element.
4. *x2*: An integer which represents the x-index of the second element.
5. *y2*: An integer which represents the y-index of the second element.

Return value:
Returns *None*

In [9]:
# FUNCTION TO SWAP TWO POSITIONS IN A 2D MATRIX

def swap(state, x1, y1, x2, y2):
    temp = state[matrix_to_list_8p(x1, y1)] # ASSIGN FIRST INDEX VALUE TO TEMPORARY VARIABLE
    state[matrix_to_list_8p(x1, y1)] = state[matrix_to_list_8p(x2, y2)] # ASSIGN SECOND INDEX VALUE TO FIRST POSITION
    state[matrix_to_list_8p(x2, y2)] = temp # ASSIGN TEMPORARY VARIABLE'S VALUE TO SECOND POSITION
    return state

#### goalTestF_8p <a id = 'goalTestF_8p'></a>
This function is for check ing if a given state is the required goal state. It takes in *state* and *goalState*, both of which are lists, and checks for equality between them.<br>
Parameters: 
1. *state*: State of the puzzle, represented as a list with 9 elements.
2. *goalState*: A list with 9 elements which represents the goal state of the 8-puzzle.

Return value: A boolean. True if *state* is equal to *goalState*, False if otherwise.

In [10]:
def goalTestF_8p(state, goalState):
    return state == goalState

#### Node class
The *Node* class is essentially used to store information associated with a node in the search tree. This information includes the state, f, g and h values of the node. 
<a id = 'as'></a>

In [11]:
class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"


#### aStarSearch <a id = 'aStarSearch'></a>
The following cell contains the implementation of A\*. *aStarSearch* initiates the recursive call, by calling the helper function aStarSearchHelper. It has the following parameters: <br>
1. startState: A list with 9 elements which represents the starting state of the 8-puzzle
2. actionsF: A functions which returns all valid actions from a state.
3. takeActionF: A function which applies a given action onto a given state, and returns the new state of the eight-puzzle.
4. goalTestF: A function which checks if a given state is the goal state or not and returns the appropriate boolean value.
5. hF: The heuristic function to be used by the algorithm.

Returns: The result from calling the helper function aStarSearchHelper.

The helper function *aStarSearchHelper* has the following parameters:
1. parentNode: An object of *Node* class.
2. actionsF: A functions which returns all valid actions from a state.
3. takeActionF: A function which applies a given action onto a given state, and returns the new state of the eight-puzzle.
4. goalTestF: A function which checks if a given state is the goal state or not and returns the appropriate boolean value.
5. hF: The heuristic function to be used by the algorithm.
6. fmax: The maximum f value of the nodes upto which the search is carried out.

Returns: The solution path.

In [14]:
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))


def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global nodes
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        nodes = nodes + 1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":             
            result.insert(0,parentNode.state)     
            return (result, bestChild.f)

## 3. Heuristic Functions and Admissibility <a id = 'heuristic'></a>

### What is a heuristic?
Merriam-Webster defines 'heuristic' as "involving or serving as an aid to learning, discovery, or problem-solving by experimental and especially trial-and-error methods" (source: https://www.merriam-webster.com/dictionary/heuristic).

In the context of informed search, the heuristic function essentially adds to the 'informed' part of informed search. As indicated above, the heuristic function h(n) should be the estimated cost of the cheapest path from a given node n to the goal.

### Admissibility
In order to obtain an optimal solution path, every heuristic must satisfy a certain rudimentary requirement, which is admissibility. An admissible heuristic is one which **never overestimates the true cost of the solution path**. In the words of our textbook authors, "admissible heuristics are by nature optimistic because they think the cost of solving
the problem is less than it actually is" (Russel and Norvig, 3.5.2).    

In this Notebook, we shall use the heuristic functions outlined in the following cells.

#### 1. h1_8p
This heuristic function returns 0 for all states including the goal state. Since it's constant for all states, it doesn't really help inform the algorithm of where the goal might be, however, it's still valid since it is admissible (0 can in no way be an overestimate of cost). It can be considered as a very 'stupid' heuristic.  

In [15]:
def h1_8p(state, goalState):
    return 0

#### 2. h2_8p
This heuristic function takes a state and the goal state as parameters, and returns the **Manhattan Distance** that the blank in the start state is from the goal state. Manhattan Distance is the distance between two points measured along axes at right angles. This can be computed by simply finding the sum of the horizontal and vertical components of the distance between the two blank locations. This sum is returned by the function. <br>

<img src = "http://apprize.info/game/algorithms/algorithms.files/image207.jpg", height = 550, width = 550>
Source: http://apprize.info/game/algorithms/algorithms.files/image207.jpg

### Is h2_8p Admissible?
Yes it is. The Manhattan Distance implemented in this function is basically the shortest possible path that can be taken to reach the goal state. No matter what other moves the other tiles make, it is definitely necessary to get the blank to move from start state position to the position in the goal state.<br>
Since tiles can only move either left, right, up or down, the shortest possible path that can be taken by a tile between any two points in the eight-puzzle is through the horizontal and vertical components of the straight-line Euclidean distance, a.k.a. the Manhattan distance. Such a path yeilds the absolute minimum path cost, and therefore can in no way overestimate the solution path cost. Hence, h2_8p is indeed admissible.  

In [17]:
# Heuristic function which computes the Manhattan Distance of the blank
def h2_8p(state, goalState):
    x1, y1 = findBlank_8p(state)
    x2, y2 = findBlank_8p(goalState)
    x = abs(x1 - x2)
    y = abs(y1 - y2)
    return x + y

#### 3. h3_8p
This heuristic function also uses Manhattan Distance to estimate solution path cost, but this differs from h2_8p in that, the Manhattan Distance calculation implemented in this function conforms with what is defined by our textbook authors (see section 3.6, 'Artificial Intelligence - A Modern Approach' by Russell and Norvig). In h2_8p, we used the distance the blank was from the goal position, but in this function, we use the sum of the distances of each tile from their respective goal positions. I am compelled to clarify that 'distance' here refers to the sum of the horizontal and vertical components of the Euclidean distance between two points on the eight-puzzle. We shall discuss the admissibility of this heuristic in the cells below.   

### Is h3_8p Admissible?
In h2_8p, we considered the absolute minimum path cost that could be generated, to be the 'distance' (see previous cell) of the blank alone, from it's position in the start state to that in the goal state. Let's generalize this a bit further. We can consider a case of absolute minimum path cost also to occur when the only possible moves that can be made by each tile are one step closer to the goal, i.e., one step towards their respective positions in the goal state. Therefore, this heuristic is also admissible. <br>

It has to be taken note of, however, that we DO NOT include the distance calculation for the blank. If we do so, then the heuristic is no longer admissible. This can be observed from the fact that when a tile is moved, **the blank position changes accordingly as well**. So we can think of the blank's moves also being considered when the distance calculation is performed for each tile, and therefore, don't have to include it's distance calculation.

In [18]:
# H3 - Actual Manhattan distance
def h3_8p(state, goalState):
    dist = 0
    for i in range(9):
        if state[i] == 0:
            continue
        # Find element at position i in state
        element = state[i]
        # Find x and y of this element
        x1, y1 = list_to_matrix_8p(i)
        # Index of same element in goalState
        xg, yg = list_to_matrix_8p(goalState.index(element))
        # Compute sum
        x = abs(x1 - xg)
        y = abs(y1 - yg)
        #print(x, y)
        s = x + y
        #print(s)
        dist = s + dist
    return dist

## 4. Iterative Deepening Search

The following cells contain the implementation for Iterative Deepening Search (IDS). Iterative deepening is a graph/tree search algorithm in which a depth-limited depth-first search is carried out repeatedly, each time with increasing depth limits. Every time a depth-limited search iteration is completed (and the goal is not found yet), all the nodes explored so far are discarded to save-up memory, and depth-first search is carried out again after increasing the depth-limit by one. 

The *depthLimitedSearch* function uses depth-limited depth-first search to try and get to the goal state. The parameters *actionsF* and *takeActionF* are functions which basically define the type of puzzle or problem which is to be solved through this function. <br>
In the following implementation, *depthLimitedSearch* is a recursive function. The *actionsF* function is used to get all the valid actions which can be applied to the current state, and the child nodes (possible actions from a state) are generated using the *takeActionF* function. <br>
It takes the following parameters:
1. *state*: State of the puzzle, represented as a list with 9 elements.
2. *goalState*: A list which represents the goal state of the puzzle.
3. *actionsF*: A function which, when given the state, returns a list of valid actions from that state.
4. *takeActionF*: A function which, when given a state and an action, applies the action onto the state and returns the new state of the puzzle.
5. *depthLimit*: An integer which designates the depth upto which the depth-limited depth-first search can go.

For each child state, *depthLimitedSearch* makes a recursive call to itself to descend down each branch of the search-tree. In each recursive call, the depth-limit parameter is decremented by 1, so that, if the depth limit is reached, and the goal hasn't been found, a conditional statement is triggered, returning the string 'cutoff', signifying that the goal state could not be found within the depth limit specified. If the goal could not be found in the entire tree, we return 'failure'.
If the goal is found, then we insert the child state in front of the *result* list, and return it i.e. we keep constructing the solution path on *result*.<br>
This function returns a list containing the solution path, that is, a list containing the states that the puzzle had to go through to get to the goal state. If the goal state cannot be found within the depth specified by *depthLimit*, then 'cutoff' is returned. If the goal state cannot be found in the entire search-tree, the string 'failure' is returned.

In [203]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    global nodes
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        childState, _ = takeActionF(state, action)
        nodes = nodes + 1
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit - 1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result != 'failure':
            result.insert(0, childState)
            return result 
    if cutoffOccurred:
        return 'cutoff' # RETURN 'cutoff' IF NO GOAL COULD BE FOUND WITHIN depthLimit
    else:
        return 'failure' # RETURN 'failure' IF NO GOAL STATE COULD BE FOUND IN THE ENTIRE SEARCH-TREE

The iterativeDeepeningSearch function iteratively calls depthLimitedSearch for each depth value from 0 to maxDepth. Once the goal state is found, it inserts startState to the front of result and returns it (the solution path).

In [ ]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    depth = 0
    while(depth < maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result.insert(0, startState)
            return result
        depth += 1
    return 'cutoff'

# EBF

In [204]:
def ebf(nNodes, depth, precision = 0.00001):
	left = 1
	right = nNodes
	if nNodes == 0:
		return 0
	b = 1.0
	while(True):
		if abs(right - left) <= precision:
			break
		#print('left', left)
		#print('right', right)
		b = (left + right)/2
		#print('b', b)
		n = (pow(b, (depth+1)) - 1)/(b - 1)
		if nNodes < n:
			right = b - precision
		elif nNodes > n:
			left = b + precision
		else:
			return b
	return b    

# 8 puzzle

## A star

In [205]:
#startState = [1, 0, 3, 4, 2, 6, 7, 5, 8]
#goalState = [1, 2, 3, 4, 5, 6, 7, 8, 0]

#startState = [1,2,3,4,0,5,6,7,8]
#goalState = [3,5,0,2,1,8,4,7,6]

startState = [1,2,3,4,0,5,6,7,8]
#goalState = [1,2,3,4,0,5,6,7,8]
goalState = [1,2,3,4,5,8,6,0,7]
#goalState = [1,0,3,4,5,8,2,6,7]

aStarSearch(startState, actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, goalState), lambda s: hf(s, goalState))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 5, 8, 6, 7, 0],
  [1, 2, 3, 4, 5, 8, 6, 0, 7]],
 3)

## IDS

In [206]:
iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 10)

[[1, 2, 3, 4, 0, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 0, 6, 7, 8],
 [1, 2, 3, 4, 5, 8, 6, 7, 0],
 [1, 2, 3, 4, 5, 8, 6, 0, 7]]

First, some example output for the ebf function.

In [207]:
ebf(10, 3)

1.6608019390487667

The smallest argument values should be a depth of 0, and 1 node.

In [208]:
ebf(1, 0)

1.0

In [209]:
ebf(2, 1)

1.0000052590942383

In [210]:
ebf(2, 1, precision=0.000001)

1.0000009073524474

In [211]:
ebf(200000, 5)

11.275616400465465

In [212]:
ebf(200000, 50)

1.2347985300998876

Here is a simple example using our usual simple graph search.

In [213]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [214]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [215]:
takeActionF_simple('a', actions[0])

('b', 1)

In [216]:
goalTestF_simple('a', 'a')

True

In [217]:
h_simple('a', 'z')

1

In [218]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [219]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

In [220]:
import pandas as pd
import numpy as np
import tabulate
from IPython.display import display, HTML

from IPython.display import display_html
def disp(dataframes):
    html_str=''
    for i in dataframes:
        html_str = html_str + i
    display_html(html_str.replace('table','table style="display:inline; text-align:center"'),raw=True)

def runExperiment(goalState1, goalState2, goalState3, hList = []):
    global nodes
    row1 = ['']
    row2 = ['Algorithm']
    row3 = ['IDS']
    rowi = []
    # Iterate through list of heuristic functions and append name first.
    for h in range(len(hList)+1):
        if h == 0:
            continue
        rowi.append(['A*h' + str(h)])

    mas = []
    final = [row1, row2, row3]#, row4, row5, row6]
    for r in rowi:
        final.append(r)
    t = tabulate.tabulate(final, stralign = 'center', tablefmt = 'html')

    mas.append(t)
    for r in rowi:
        r.clear()
    startState = [1,2,3,4,0,5,6,7,8]
    goalStateList = [goalState1, goalState2, goalState3]
    for goalState in goalStateList:
        row1 = ['', goalState, '']
        row2 = ['Depth', 'Nodes', 'EBF']
        row3 = []
        maxd = float('inf')
        ids = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, maxd)
        dids = len(ids) - 1
        n = nodes
        ebfids = ebf(n, dids)
        ebfids = "{:.3f}".format(ebfids) 
        row3.append(dids)
        row3.append(n)
        row3.append(ebfids)
        nodes = 0

        for i in range(len(hList)):
            hf = hList[i]
            ah = aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda s: goalTestF_8p(s, goalState), lambda s: hf(s, goalState))
            d = ah[1]
            n = nodes
            ebf1 = ebf(n, d)
            ebf1 = "{:.3f}".format(ebf1)
            rowi[i].append(d)
            rowi[i].append(n)
            rowi[i].append(ebf1)
            nodes = 0

        final = [row1, row2, row3]
        for row in rowi:
            final.append(row)
        t = tabulate.tabulate(final, stralign = 'center', tablefmt = 'html')
        for row in rowi:
            row.clear()
        mas.append(t)
    disp(mas)

In [221]:
goalState1 = [1,2,3,4,0,5,6,7,8]
goalState2 = [1,2,3,4,5,8,6,0,7]
goalState3 = [1,0,3,4,5,8,2,6,7]
#goalState3 = [1,2,3,4,0,5,6,7,8]
startState = [1,2,3,4,0,5,6,7,8]
runExperiment(goalState1, goalState2, goalState3, [h1_8p, h2_8p, h3_8p])

Algorithm 
 IDS 
 A*h1 
 A*h2 
 A*h3 
 
 
 
 [1, 2, 3, 4, 0, 5, 6, 7, 8] 
 Depth Nodes EBF 
 0 237 237.000 
 0 0 0.000 
 0 0 0.000 
 0 0 0.000 
 
 
 
 [1, 2, 3, 4, 5, 8, 6, 0, 7] 
 Depth Nodes EBF 
 3 43 3.086 
 3 116 4.488 
 3 51 3.297 
 3 9 1.578 
 
 
 
 [1, 0, 3, 4, 5, 8, 2, 6, 7] 
 Depth Nodes EBF 
 11 225850 2.954 
 11 643246 3.263 
 11 100046 2.733 
 11 1172 1.762

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [222]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            